<a href="https://colab.research.google.com/github/Bagito999/Deep-Learning/blob/main/Chapter_13_Teori.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Chapter 13 - Loading and Preprocessing Data with TensorFlow

##  Tujuan Bab

Bab ini membahas cara:
- Memuat data efisien menggunakan `tf.data`
- Melakukan preprocessing secara modular dan scalable
- Mengoptimalkan input pipeline untuk performa maksimum

---

##  Tantangan dalam Input Data Pipeline

1. Dataset besar tidak bisa dimuat sekaligus ke memori
2. Perlu preprocessing yang cepat dan konsisten
3. Training efisien membutuhkan pipeline input yang teroptimasi

---

##  API `tf.data`

TensorFlow menyediakan API `tf.data` untuk:
- Membuat objek `Dataset`
- Melakukan transformasi data
- Menangani pipeline streaming (lazy evaluation)

### 🔹 Dataset Creation

Beberapa cara umum:
- Dari array: `from_tensor_slices()`
- Dari file CSV/TFRecord: `TextLineDataset`, `TFRecordDataset`
- Dari generator Python

---

##  Transformasi Dataset

`Dataset` bersifat **lazy**, artinya operasi hanya dilakukan saat dibutuhkan.

Transformasi umum:
- `map()`: transformasi elemen
- `filter()`: seleksi elemen
- `batch()`: kelompokkan data
- `shuffle()`: acak elemen
- `repeat()`: ulangi dataset

---

##  Optimisasi Performance

###  Prefetching

`prefetch(n)` memungkinkan loading data dan training berjalan paralel.

###  Interleaving & Parallel Mapping

- `interleave()`: membaca beberapa file secara paralel
- `map(..., num_parallel_calls=tf.data.AUTOTUNE)`: paralelisasi preprocessing

###  Caching

`cache()` menyimpan hasil transformasi agar tidak dihitung ulang setiap epoch.

---

##  Format Data: TFRecord

Format biner efisien untuk data besar:
- Kompresi tinggi
- Dibaca dengan cepat
- Digunakan luas di produksi (Google, TFX)

Biasanya digunakan bersama dengan:
- `TFRecordDataset`
- `tf.train.Example` (untuk parsing protokol buffer)

---

##  TFRecord dan Protobuf

### Struktur Record:
```protobuf
features {
  feature {
    key: "feature_name"
    value: { float_list / int64_list / bytes_list }
  }
}
```

##  Parsing dan Decoding

Data perlu di-*decode* sesuai tipe dan struktur aslinya. Setelah data dibaca (misalnya dari file TFRecord), TensorFlow perlu mengetahui bagaimana menafsirkan byte tersebut.

### Contoh decoding umum:

- **Gambar**:
  - JPEG → `tf.io.decode_jpeg()`
  - PNG → `tf.io.decode_png()`
- **Teks/Bytes**:
  - → `tf.io.decode_raw()`
- **Label numerik**:
  - → `tf.cast()` ke `tf.int64` atau `tf.float32`
- **Parsing TFRecord**:
  - Gunakan `tf.io.parse_single_example(serialized_example, feature_description)`

### 🔹 Feature Description

Untuk mem-parsing TFRecord, deskripsi fitur dibutuhkan:
```python
feature_description = {
  'image': tf.io.FixedLenFeature([], tf.string),
  'label': tf.io.FixedLenFeature([], tf.int64)
}
```

##  Preprocessing Data

Transformasi umum yang dilakukan sebelum data masuk ke model:

- **Normalisasi**: skala nilai ke rentang stabil (misalnya 0–1)
- **Standarisasi**: kurangi nilai rata-rata, bagi dengan standar deviasi
- **Resizing**: menyesuaikan ukuran input (misalnya gambar ke 224x224)
- **Augmentasi Data**: rotasi, flipping, cropping, zoom untuk meningkatkan generalisasi
- **Tokenisasi**: memecah teks menjadi kata/karakter dan mengubahnya ke indeks
- **Padding**: menyesuaikan panjang teks agar seragam untuk batch processing

###  Pilihan Lokasi Preprocessing:

1. **Offline Preprocessing**  
   - Lakukan sebelum training dan simpan ke disk (file .npy, TFRecord, dll)

2. **In-pipeline Preprocessing (tf.data)**  
   - Gunakan fungsi `map()` dalam pipeline data

3. **In-model Preprocessing**  
   - Gunakan layer seperti `tf.keras.layers.Rescaling`, `Resizing`, `TextVectorization`

---

##  Bandingkan: Preprocessing di Model vs Dataset

| Lokasi Preprocessing   | Keunggulan                         | Kelemahan                            |
|------------------------|------------------------------------|--------------------------------------|
| **Di dalam model**     | Portable, dapat di-deploy bersama model | Lebih lambat saat inferensi            |
| **Dalam tf.data pipeline** | Cepat dan fleksibel selama training    | Tidak otomatis dibawa saat ekspor model |
| **Offline preprocessing** | Konsisten, bisa didistribusi       | Tidak fleksibel terhadap perubahan    |

---

##  Mengatur Pipeline untuk Performance Maksimal

Agar training efisien, ikuti urutan transformasi berikut:

```text
Load → Shuffle → Cache → Batch → Prefetch
```

##  TFX: TensorFlow Extended

**TFX (TensorFlow Extended)** adalah platform end-to-end produksi machine learning yang dirancang untuk membangun dan mengelola pipeline ML berbasis TensorFlow dalam skala besar.

TFX digunakan untuk:
- Persiapan data
- Pelatihan model
- Evaluasi model
- Deployment model ke produksi

###  Komponen Utama TFX:

- **ExampleGen**  
  Komponen pertama yang mengimpor data dari berbagai sumber dan mengubahnya menjadi format standar TFRecord.

- **StatisticsGen**  
  Menghitung statistik data seperti distribusi, mean, standar deviasi, missing values, dll.

- **SchemaGen**  
  Menganalisis statistik dan menghasilkan skema fitur yang menjelaskan jenis dan batasan data.

- **ExampleValidator**  
  Mendeteksi anomali, missing values, dan outlier berdasarkan skema.

- **Transform**  
  Menerapkan preprocessing skala besar dengan TensorFlow Transform (TFT), sehingga preprocessing dapat digunakan saat training dan inferensi.

- **Trainer**  
  Melatih model TensorFlow menggunakan arsitektur Keras atau Estimator.

- **Tuner (opsional)**  
  Untuk hyperparameter tuning otomatis.

- **Evaluator**  
  Mengevaluasi model untuk memilih versi terbaik berdasarkan metrik (accuracy, fairness, dll).

- **InfraValidator**  
  Memastikan model dapat di-*serve* sebelum benar-benar dikirim ke produksi.

- **Pusher**  
  Menyimpan model akhir dan mengirimnya ke sistem serving seperti TensorFlow Serving.

---

##  Rumus dan Konsep Kunci

###  Normalisasi

Menstabilkan dan menskalakan input:

\[
$x_{\text{norm}} = \frac{x - \mu}{\sigma}$
\]

- \($ x $\): nilai asli
- \($ \mu $\): rata-rata
- \($ \sigma $\): deviasi standar

Digunakan untuk mencegah dominasi fitur dengan skala besar terhadap model.

---

###  Shuffle Buffer Size

Buffer digunakan saat proses `shuffle()` untuk mengacak data sebelum batching.

- **Buffer besar** → pengacakan lebih acak, tetapi butuh memori lebih besar.
- Ideal: `buffer_size >= dataset size` jika memori memungkinkan.

---

###  Pipeline Optimisasi

**Urutan pipeline optimal**:

```text
Load → Shuffle → Cache → Batch → Prefetch
```

###  Pipeline dengan tf.data — Optimasi Performa

Gunakan kombinasi transformasi berikut untuk membuat pipeline data yang efisien:

- `shuffle(buffer_size)`  
  Mengacak urutan data agar training tidak terpengaruh pola data statis.  
  Buffer size menentukan seberapa acak pengocokan data dan berapa banyak memori yang digunakan.

- `cache()`  
  Menyimpan hasil transformasi dataset di memori atau disk setelah pertama kali diproses.  
  Hal ini menghindari pengulangan preprocessing setiap epoch, sehingga training menjadi lebih cepat.

- `batch(batch_size)`  
  Mengelompokkan data menjadi batch agar dapat diproses secara paralel selama training.  
  Batch size yang lebih besar meningkatkan efisiensi GPU, namun perlu disesuaikan dengan kapasitas memori.

- `prefetch(tf.data.AUTOTUNE)`  
  Melakukan *overlapping* antara loading data di CPU dan training di GPU.  
  `AUTOTUNE` secara otomatis memilih jumlah buffer yang optimal.

###  Urutan Pipeline Ideal

```python
dataset.shuffle(buffer_size)
       .cache()
       .batch(batch_size)
       .prefetch(tf.data.AUTOTUNE)
```

##  Kesimpulan

- **`tf.data` API** menyediakan cara yang efisien, modular, dan scalable untuk membangun pipeline data di TensorFlow, cocok untuk dataset kecil hingga sangat besar.

- Gunakan transformasi penting dalam pipeline seperti:
  - `shuffle()` untuk mencegah overfitting urutan data
  - `cache()` untuk menghindari komputasi berulang
  - `batch()` untuk pelatihan paralel
  - `prefetch()` untuk overlapping CPU dan GPU

- **TFRecord** adalah format penyimpanan data yang efisien dan cocok untuk produksi, terutama untuk training skala besar dan distribusi.

- **Preprocessing** dapat dilakukan:
  - **Offline**: efisien untuk data statis
  - **Dalam pipeline `tf.data`**: fleksibel dan cocok untuk eksperimen
  - **Di dalam model**: portable dan otomatis dibawa saat deploy

- Gunakan **TFX (TensorFlow Extended)** untuk pipeline ML produksi end-to-end:
  - Menyediakan komponen untuk ingest data, preprocessing, training, evaluasi, dan deployment.

- Pipeline yang dioptimalkan akan:
  - Mengurangi waktu epoch secara drastis
  - Mengurangi penggunaan CPU/GPU yang tidak efisien
  - Memastikan proses pelatihan stabil, konsisten, dan scalable

## Referensi
Géron, A. (2019). *Hands-On Machine Learning with Scikit-Learn, Keras, and TensorFlow*. O'Reilly Media.